In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Parameters
SPEED_THRESHOLD = 4  # m/s
MIN_STOP_DURATION = 1.0  # seconds
DIST_THRESH = 2.5
EXPORT_INTERVAL = 0.16  # seconds
PED_BOX = (0.3, 0.3)
TRACTOR_BOX = (3.4,1.8)         

# ----------------------------------
# Helpers
# ----------------------------------

def filter_stopped_vehicles(df, speed_col='speed', time_col='TimeStamp'):
    """Filter out stopped vehicles longer than threshold"""
    df = df.copy()
    df['time_diff'] = df.groupby('Track ID')[time_col].diff()
    df['is_stopped'] = df[speed_col] < SPEED_THRESHOLD
    stop_groups = (df['is_stopped'] != df['is_stopped'].shift()).cumsum()
    df['stop_duration'] = df.groupby(['Track ID', stop_groups])['time_diff'].cumsum()
    return df[~((df['is_stopped']) & (df['stop_duration'] >= MIN_STOP_DURATION))].drop(columns=['time_diff', 'is_stopped', 'stop_duration'])

def get_rotated_corners(x, y, heading, length, width):
    """Return coordinates of rotated bounding box corners"""
    half_l, half_w = length / 2, width / 2
    corners = np.array([
        [ half_l,  half_w],
        [ half_l, -half_w],
        [-half_l, -half_w],
        [-half_l,  half_w]
    ])
    rad = np.radians(heading)
    rot = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad),  np.cos(rad)]
    ])
    return (corners @ rot.T) + np.array([x, y])

def get_closest_corners(row):
    """Get direction vector and min distance between closest corners"""
    ped_corners = get_rotated_corners(row['x_smooth_ped'], row['y_smooth_ped'], row['HA_ped'], *PED_BOX)
    tractor_corners = get_rotated_corners(row['x_smooth_tractor'], row['y_smooth_tractor'], row['HA_tractor'], *TRACTOR_BOX)
    dists = cdist(ped_corners, tractor_corners)
    idx = np.unravel_index(np.argmin(dists), dists.shape)
    min_dist = dists[idx]
    direction_vec = tractor_corners[idx[1]] - ped_corners[idx[0]]
    return min_dist, direction_vec

def calculate_attc(row):
    """Compute ATTC using projection of relative motion on direction vector"""
    min_dist, direction_vec = get_closest_corners(row)
    norm = np.linalg.norm(direction_vec)
    if norm == 0:
        return np.inf
    unit_vec = direction_vec / norm

    rel_v = np.array([
        row['vx_smooth_ped'] - row['vx_smooth_tractor'],
        row['vy_smooth_ped'] - row['vy_smooth_tractor']
    ])
    rel_a = 0.5 * np.array([
        row['ax_ped'] - row['ax_tractor'],
        row['ay_ped'] - row['ay_tractor']
    ])

    closing_rate = -np.dot(rel_v, unit_vec) + np.dot(rel_a, unit_vec)
    return min_dist / closing_rate if closing_rate > 0 else np.inf

# ----------------------------------
# Load Data
# ----------------------------------

df_ped = pd.read_csv(r"D:\T\test_codeEVT\nd\ped_smooth.csv")
df_tractor = pd.read_csv(r"D:\T\test_codeEVT\nd\tractor_smooth.csv")
df_tractor['speed'] = np.sqrt(df_tractor['vx_smooth']**2 + df_tractor['vy_smooth']**2)

# Filter stopped vehicles
print(f"Original tractorrcycle count: {len(df_tractor)}")
df_tractor = filter_stopped_vehicles(df_tractor, speed_col='speed')
print(f"Filtered tractorrcycle count: {len(df_tractor)}")
print(df_tractor['speed'].mean())
# # Calculate yaw rates
# for df in [df_ped, df_tractor]:
#     df['yaw_rate'] = df.groupby('Track ID').apply(
#         lambda x: x['HA'].diff() / x['TimeStamp'].diff()
#     ).reset_index(level=0, drop=True)

# Round timestamps
df_ped['Time_rounded'] = (df_ped['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL
df_tractor['Time_rounded'] = (df_tractor['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL


Original tractorrcycle count: 153
Filtered tractorrcycle count: 38
4.039249118814901


In [3]:

# ----------------------------------
# Merge and Process
# ----------------------------------

merged = pd.merge(
    df_ped, 
    df_tractor, 
    on='Time_rounded', 
    suffixes=('_ped', '_tractor')
)

merged['Center_dist'] = np.hypot(
    merged['x_smooth_ped'] - merged['x_smooth_tractor'],
    merged['y_smooth_ped'] - merged['y_smooth_tractor']
)

# Filter by center threshold
results = merged[merged['Center_dist'] <= DIST_THRESH].copy()

# Calculate ATTC using corrected method
results['ATTC'] = results.apply(calculate_attc, axis=1)
results[['mindis', 'direction_vec']] = results.apply(
    lambda row: pd.Series(get_closest_corners(row)),
    axis=1
)
# Output
output = results[['Track ID_ped', 'Track ID_tractor', 'TimeStamp_ped', 'ATTC']].rename(columns={
    'Track ID_ped': 'Ped_ID',
    'Track ID_tractor': 'tractor_ID',
    'TimeStamp_ped': 'TimeStamp'
})
print(output)
len(results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_tractor']].drop_duplicates())

    Ped_ID  tractor_ID  TimeStamp       ATTC
40   14419       14474    4954.16  10.311933
41   14419       14474    4954.32        inf
42   14419       14474    4954.48   0.219135
43   14419       14474    4954.64   0.275010
44   14419       14474    4954.80        inf
45   14419       14474    4954.96        inf
46   14419       14474    4955.12        inf
47   14419       14474    4955.28   2.584048
48   14419       14474    4955.44   3.027749
49   14419       14474    4955.60        inf


1

In [4]:
results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_tractor']].drop_duplicates()

,Track ID_ped,Track ID_tractor
42,14419,14474


In [5]:
results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_tractor','TimeStamp_ped']]

,Track ID_ped,Track ID_tractor,TimeStamp_ped
42,14419,14474,4954.48


In [6]:
# REJECTED

In [7]:
# Create new DataFrame with selected and renamed columns
conflict_summary = results[[
    'Track ID_ped',        # Pedestrian ID
    'Track ID_tractor',       # Vehicle ID
    'Type_tractor',           # Vehicle type
    'TimeStamp_ped',       # Timestamp
    'mindis',              # Minimum corner distance
    'ATTC'                 # ATTC value
]].copy()

# Rename columns
conflict_summary.rename(columns={
    'Track ID_ped': 'PED_ID',
    'Track ID_tractor': 'VEH_ID',
    'Type_tractor': 'TYPE',
    'TimeStamp_ped': 'TIMESTAMP',
    'mindis': 'MIN_COR_DIS',
    'ATTC': 'ATTC'
}, inplace=True)

# Preview
print(conflict_summary.head())


    PED_ID  VEH_ID      TYPE  TIMESTAMP  MIN_COR_DIS       ATTC
40   14419   14474   Tractor    4954.16     0.426154  10.311933
41   14419   14474   Tractor    4954.32     0.418333        inf
42   14419   14474   Tractor    4954.48     0.641229   0.219135
43   14419   14474   Tractor    4954.64     1.103138   0.275010
44   14419   14474   Tractor    4954.80     1.567735        inf


In [8]:
conflict_summary.to_csv(r"D:\T\test_codeEVT\ATTC_Data/Tractor_Ped.csv", index=False)